In [11]:
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [12]:
dataset = load_dataset("mteb/tweet_sentiment_extraction")


In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.fc = nn.Linear(1024, 1024)

    def forward(self, x):
        x=x.float()
        output = self.fc(x)
        return output.long()


In [15]:
class CombinedModel(nn.Module):
    def __init__(self, transformer_model):
        super(CombinedModel, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.transformer_model = transformer_model

    def forward(self, input_ids, attention_mask, labels):
        
        
        # Transformerモデルの出力を取得
        transformer_outputs = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # features = self.feature_extractor(transformer_outputs)
        
        return transformer_outputs

combined_model = CombinedModel(model)


In [16]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [17]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [18]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [19]:
training_args = TrainingArguments(
    output_dir="test_trainer2",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=10,
)

trainer = Trainer(
    model=combined_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [20]:
print("hello")
trainer.train()

hello


Step,Training Loss


TrainOutput(global_step=300, training_loss=0.7026987711588542, metrics={'train_runtime': 228.8801, 'train_samples_per_second': 13.107, 'train_steps_per_second': 1.311, 'total_flos': 0.0, 'train_loss': 0.7026987711588542, 'epoch': 3.0})

In [21]:
import evaluate

trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.06 GiB. GPU 0 has a total capacity of 24.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 52.98 GiB is allocated by PyTorch, and 392.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)